In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import make_scorer, matthews_corrcoef
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

In [2]:
df=pd.read_csv('./res/df_inne_dropped')

In [3]:
types_of_categories={}
ordered=['Powietrze','Media_k','Gry_czas']

In [4]:
###dla atrybutow porzadkowych
print(df['Powietrze'].value_counts())

Powietrze
Codziennie               15
Kilka razy w tygodniu    13
Rzadko                    1
Name: count, dtype: int64


In [5]:
from copy import deepcopy
df_encoded=deepcopy(df)
encoder_powietrze=LabelEncoder()
ordered=['Rzadko','Kilka razy w tygodniu','Codziennie']

def ordered_encoding(data:pd.DataFrame,etyquete_dict:dict)->list:
    #label_dict={}
    new_vals=[]
    #for idx,elem in enumerate(order):
     #   label_dict[elem]=idx
    #print(label_dict)
    #print(data.values.flatten())
    #for elem in data.values.flatten() :
     #   new_vals.append(label_dict[elem])
    #return new_vals
    for elem in data.values.flatten():
        new_vals.append(etyquete_dict[elem])
    return new_vals

In [6]:
####dla 3 kolumn ktore podmienilismy mamy gotowe kody, i nie trzeba samemu wpisywac wartosci
colname='Powietrze' ## wpisujemy kolumne do sprawdzenia
print(df[colname])


0                Codziennie
1                Codziennie
2                Codziennie
3     Kilka razy w tygodniu
4                Codziennie
5     Kilka razy w tygodniu
6                Codziennie
7     Kilka razy w tygodniu
8     Kilka razy w tygodniu
9                Codziennie
10               Codziennie
11                   Rzadko
12               Codziennie
13               Codziennie
14    Kilka razy w tygodniu
15    Kilka razy w tygodniu
16               Codziennie
17               Codziennie
18    Kilka razy w tygodniu
19               Codziennie
20               Codziennie
21    Kilka razy w tygodniu
22    Kilka razy w tygodniu
23               Codziennie
24    Kilka razy w tygodniu
25               Codziennie
26    Kilka razy w tygodniu
27    Kilka razy w tygodniu
28    Kilka razy w tygodniu
Name: Powietrze, dtype: object


In [7]:

etiquete_dict={'Rzadko':1,'Kilka razy w tygodniu':4,'Codziennie':7} ###przypisujemy zakodowanie do wartosci. tylko atrbuty porzadkowe!!!!!1
df[colname]=ordered_encoding(df[colname], etiquete_dict)
print(df[colname])

0     7
1     7
2     7
3     4
4     7
5     4
6     7
7     4
8     4
9     7
10    7
11    1
12    7
13    7
14    4
15    4
16    7
17    7
18    4
19    7
20    7
21    4
22    4
23    7
24    4
25    7
26    4
27    4
28    4
Name: Powietrze, dtype: int64


In [8]:
print(df['Media_k'].value_counts())
df['Media_k']=ordered_encoding(df['Media_k'],etiquete_dict)
print(df['Media_k'].value_counts())

Media_k
Codziennie               20
Kilka razy w tygodniu     6
Rzadko                    3
Name: count, dtype: int64
Media_k
7    20
4     6
1     3
Name: count, dtype: int64


In [9]:
etiquete_dict={'Rzadko':0,'Raz w tygodniu':1,'Kilka razy w tygodniu':4,'Codziennie':7}

print(df['gry_czas'].value_counts())
df['gry_czas']=ordered_encoding(df['gry_czas'],etiquete_dict)
print(df['gry_czas'].value_counts())

gry_czas
Rzadko                   12
Kilka razy w tygodniu     9
Codziennie                5
Raz w tygodniu            3
Name: count, dtype: int64
gry_czas
0    12
4     9
7     5
1     3
Name: count, dtype: int64


In [10]:
print(df.columns)

Index(['Moda', 'Muzyka', 'Film', 'Sport_u', 'Sport_o', 'Komputer', 'Samochod',
       'Literatura', 'Smak', 'Wolny_czas', 'Miasto', 'Kraj', 'Wzrost', 'Włosy',
       'Okulary', 'Przedmiot', 'Gra', 'Wf', 'Sniadanie', 'Kino_teatr', 'Pora',
       'Ksiazki', 'Gotowanie', 'Zajecie', 'Media_u', 'Technologie',
       'Powietrze', 'Fryzura', 'Media_k', 'Paznokcie', 'Wakacje',
       'Rod_prz_sam', 'gry_czas', 'Jedzenie', 'Wartosc', 'Napoj', 'Dom',
       'Zwierzak', 'Plec'],
      dtype='object')


In [11]:
####dla atrybutu decyyzjnego

encoder=LabelEncoder()
df['Moda']=encoder.fit_transform(df['Moda'])
print(df['Moda'].value_counts())


Moda
0    13
2     8
1     8
Name: count, dtype: int64


In [12]:
print(encoder.classes_)

['Casual' 'Elegancki' 'nie dbam o mode']


In [13]:
######zmieniei WF na liczby
def stoi(string:str)->int:
    new_str=''
    for character in string:
        if character.isnumeric():
            new_str+=character
        else: 
            break
    return int(new_str)
df['Wf']=df['Wf'].apply(stoi)
print(df['Wf'])

0     2
1     0
2     1
3     2
4     1
5     2
6     1
7     1
8     1
9     2
10    2
11    0
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    0
21    0
22    0
23    2
24    1
25    1
26    1
27    1
28    2
Name: Wf, dtype: int64


In [14]:
###kolumny ktore dalej nie sa licbzowe po poprzednich zmianach kodujemy z one hot
category_columns=df.select_dtypes(include='object').columns
en_df=pd.get_dummies(df,columns=category_columns,prefix=[col+'_' for col in category_columns])
print(en_df['Moda'])


0     0
1     2
2     0
3     0
4     0
5     2
6     2
7     2
8     0
9     0
10    1
11    1
12    0
13    0
14    0
15    1
16    1
17    1
18    1
19    1
20    2
21    2
22    2
23    1
24    2
25    0
26    0
27    0
28    0
Name: Moda, dtype: int64


In [15]:
en_df.to_csv('./res/df_inne_encoded_logically_final.csv',index=False)